In [1]:
import pandas as pd
import numpy as np
import re
import emoji

### Loading the Dataset

The Facebook comments dataset can be loaded from an Excel file (e.g., ```comments.xlsx```) or a CSV file. The ```skiprows=6``` parameter is used when loading the Excel file to skip the first six rows, which contain metadata such as  comments export website details, post link, timezone information, and empty rows. However, if the dataset is in CSV format, this parameter should be omitted or set to its default value, as CSV files typically don't contain such metadata in the same structure.



In [2]:
df = pd.read_excel('/kaggle/input/fb-comments/comments.xlsx', skiprows = 6)

### Previewing the Dataset

In [3]:
df.head()

,Unnamed: 0,Unnamed: 1,Name (click to view profile),Profile ID,Date,Likes,Live video timestamp,Comment,(view source)
0,1,NaN,Haider Aljubori,100003071543537,2024-09-26 19:09:43,NaN,-,اربع نقاط من مبارة واحده. نايمين كاعدين انتم,view comment
1,2,NaN,اثير الزيدي,100010105309143,2024-09-26 19:17:25,53.0,-,انه مع نقل مباراة كوريا خارج الاردن,view comment
2,3,NaN,شاكر محمود,100026726736221,2024-09-26 19:30:07,NaN,-,أنت تسولف بالاحلام,view comment
3,4,NaN,مهند السفير,100053616079766,2024-09-26 21:20:52,NaN,-,تره بعد 8 اشهر على لعبتهم بالبصرةشنو هل المنشو...,view comment
4,5,NaN,Mohammed Nazar,61553624182189,2024-09-26 22:51:49,NaN,-,هسه انت لايمك لاحولك المن تنشر ماتكلي لخاطر ال...,view comment


### Dropping Unnecessary Columns

removing unnecessary columns from the dataset:

* **Unnamed 0:** The default name by the ```pandas``` library in the original file its the index/number of the comment
* **Unnamed 1:** Its an empty column in the original file
* **Commenter Information Name (click to view profile):** Information about the commenters (a hyperlink).
* **Comment Metadata:** Date contains the date the comment was made, Likes contains the number of likes the comment received, and Live video timestamp contains information about the live video (if applicable).

The rows to be dropped can be adjusted based on decisions regarding what data to retain and how to handle the accompanying metadata. This will be determined by the project's requirements, as defined by the team, the program coordinator, and the "IP". This is merely a demo of how we will be handling and cleaning the comments prior to annotation.

In [4]:
df = df.drop(['Unnamed: 0', 'Unnamed: 1', 'Name (click to view profile)', 'Profile ID', 'Date','Likes', 'Live video timestamp'], axis=1)

In [5]:
df.head()

,Comment,(view source)
0,اربع نقاط من مبارة واحده. نايمين كاعدين انتم,view comment
1,انه مع نقل مباراة كوريا خارج الاردن,view comment
2,أنت تسولف بالاحلام,view comment
3,تره بعد 8 اشهر على لعبتهم بالبصرةشنو هل المنشو...,view comment
4,هسه انت لايمك لاحولك المن تنشر ماتكلي لخاطر ال...,view comment


### Text Cleaning Functions

The ```is_arabic``` function checks if a given character is within the Arabic Unicode block.

The ```clean_text``` function cleans a given text by:

* **Filtering Characters:** keeping only Arabic letters, emojis, or spaces.
* **Removing Short Sentences:** removing sentences with less than ```min_length``` words.
* **Removing Empty Strings and Whitespaces:** removing empty strings and leading/trailing whitespaces.
* **Removing Emoji-only Text:** returning an empty string if the text consists only of emojis.

In [6]:

def is_arabic(char):
    # Check if the character is within the Arabic Unicode block
    return '\u0600' <= char <= '\u06FF'

def clean_text(text, min_length=1):
    # Filter characters: keep Arabic letters, emojis, or spaces
    filtered_text = ''.join([char for char in text if is_arabic(char) or char == ' ' or emoji.is_emoji(char)])

    # Remove short sentences (less than min_length)
    filtered_text = ' '.join([sentence for sentence in filtered_text.split('.') if len(sentence.split()) >= min_length])

    # Remove empty strings and leading/trailing whitespaces
    filtered_text = ' '.join([word for word in filtered_text.split() if word != '']).strip()

    # If the text consists only of emojis, return an empty string
    if all(emoji.is_emoji(char) for char in filtered_text) and len(filtered_text) > 0:
        return ''

    return filtered_text



### Cleaning the Comment Column
Applying the ```clean_text``` function to each comment in the ```Comment``` column of the dataframe

In [7]:
df['Comment'] = df['Comment'].apply(clean_text)

In [8]:
df


,Comment,(view source)
0,اربع نقاط من مبارة واحده نايمين كاعدين انتم,view comment
1,انه مع نقل مباراة كوريا خارج الاردن,view comment
2,أنت تسولف بالاحلام,view comment
3,تره بعد اشهر على لعبتهم بالبصرةشنو هل المنشورا...,view comment
4,هسه انت لايمك لاحولك المن تنشر ماتكلي لخاطر ال...,view comment
...,...,...
94,باذن الله نطيح بكوريا ونتاهل مع بعض,view comment
95,على حساباتك هي كوريا تاخذ نقطتين من ١٢ مع العر...,view comment
96,منتخب العراق صار سنين يلعب خارج ارضه وانت الكو...,view comment
97,ولكن عدم التفريط بالنقاط مع الفرق العربيه داخل...,view comment


### Removing Rows Empty Comments

In [9]:
# Drop rows where the 'comments' column is empty
df = df[df['Comment'] != '']

In [10]:
df

,Comment,(view source)
0,اربع نقاط من مبارة واحده نايمين كاعدين انتم,view comment
1,انه مع نقل مباراة كوريا خارج الاردن,view comment
2,أنت تسولف بالاحلام,view comment
3,تره بعد اشهر على لعبتهم بالبصرةشنو هل المنشورا...,view comment
4,هسه انت لايمك لاحولك المن تنشر ماتكلي لخاطر ال...,view comment
...,...,...
94,باذن الله نطيح بكوريا ونتاهل مع بعض,view comment
95,على حساباتك هي كوريا تاخذ نقطتين من ١٢ مع العر...,view comment
96,منتخب العراق صار سنين يلعب خارج ارضه وانت الكو...,view comment
97,ولكن عدم التفريط بالنقاط مع الفرق العربيه داخل...,view comment


### Exporting to the Desired Format

In [11]:
df.to_excel('cleaned_data.xlsx', index=False)
# df.to_csv('cleaned_data.csv', index = False)